In [1]:
from snowexsql.db import get_db
from snowexsql.data import ImageData, SiteData, LayerData
from rasterio.plot import show
from sqlalchemy.sql import func
import geoalchemy2.functions as gfunc
from geoalchemy2.types import Raster
from geoalchemy2.shape import to_shape
import geopandas as gpd
from snowexsql.conversions import raster_to_rasterio
from snowexsql.conversions import points_to_geopandas, query_to_geopandas, query_to_pandas
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
import pandas as pd

In [2]:
def get_bufferedPit(pit_id):
    # Distance around the pit to collect data in meters
    buffer_dist = 50

    datasets = []

    # Grab our sites details by site id
    q = session.query(LayerData).filter(LayerData.pit_id==pit_id)
    sites = q.all()

    # Grab the pit location from a single layer
    p = sites[0].geom

    # Convert the point to a pyshapely
    pit = to_shape(p)

    # Convert it to a geopandas dataframe for easy plotting
    df_pit = gpd.GeoSeries(pit)
    q = session.query(gfunc.ST_Buffer(p, buffer_dist))
    buffered_pit = q.all()[0][0]
    return buffered_pit

In [3]:
def average_amplitudes(raster):
    """
    input: 2 dimensional array of list of amplitudes over multiple rasters
    output: average amplitude over entire raster
    """
    #create a variable to average each individual raster
    single_avgs = []
    #iterate over all rasters
    for i in range(len(raster)): 
        #compute and append the mean of the raster 
        single_avgs.append(sum(dataset.read(1)[i])/len(dataset.read(1)[i]))
    
    #compute the average amplitude over all rasters
    avg_amp = sum(single_avgs)/len(single_avgs) 
    
    return avg_amp

In [20]:
# Pit Site Identifier of interest
site_name = 'Grand Mesa'

# Connect to the database we made.
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

datasets = []
amplitudes = []

# Grab our sites details by site id
#q = session.query(SiteData).filter(SiteData.site_id==site_id)
q = session.query(SiteData.site_id).filter(SiteData.site_name==site_name)
q = q.filter(SiteData.tree_canopy=="No Trees")
q = q.filter(SiteData.date>=date(2020,1,28))
q = q.filter(SiteData.date<=date(2020,3,1))
sites = q.all()
sites = [d[0] for d in sites] 
q_layer = session.query(LayerData.pit_id).filter(LayerData.site_id.in_(sites))
q_layer = q_layer.distinct()
df = query_to_pandas(q_layer, engine)
df.sort_values(by=['pit_id'],ascending=True)

# Grab the rasters, union them and convert them as tiff when done
q_raster = session.query(func.ST_AsTiff(func.ST_Union(ImageData.raster, type_=Raster)))

# Only grab rasters that are the bare earth DEM from USGS
q_raster = q_raster.filter(ImageData.type == 'insar amplitude').filter(ImageData.observers=='UAVSAR team, JPL').filter(ImageData.site_name == "Grand Mesa")
q_raster = q_raster.filter(ImageData.description.in_
                           (["Overpass Duration: 2020-02-12 16:47:20 - 2020-02-12 16:49:45 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = VV"]))

for id in df.pit_id:
    try:
        buffered_pit = get_bufferedPit(id)
        q_raster_new = q_raster.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
        rasters = q_raster_new.all()
        dataset = raster_to_rasterio(session, rasters)[0]
        avg_amplitudes = average_amplitudes(dataset.read(1))
        amplitudes.append((id,avg_amplitudes))
    except TypeError:
        #print("I failed")
        continue
#print(id)
print(len(amplitudes))
amp = pd.DataFrame(amplitudes)
amp.to_csv('amplitudes_VV2.csv')

128


In [ ]:
polarizations = ['HH','HV','VH','VV']
for i in polarizations:
    q_raster = q_raster.filter(ImageData.description.in_
                           (["Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = "+i,
                           "Overpass Duration: 2020-02-12 16:47:20 - 2020-02-12 16:49:45 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = "+i]))

    for id in df.pit_id:
        try:
            buffered_pit = get_bufferedPit(id)
            q_raster_new = q_raster.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
            rasters = q_raster_new.all()
            dataset = raster_to_rasterio(session, rasters)[0]
            avg_amplitudes = average_amplitudes(dataset.read(1))
            amplitudes.append((id,avg_amplitudes))
        except TypeError:
            #print("I failed")
            continue
    print(i)
    print(len(amplitudes))
    amp = pd.DataFrame(amplitudes)
    amp.to_csv('amplitudes_'+i+'.csv')

In [9]:
polarizations = ['HH','HV','VH','VV']
for i in polarizations:
    print("Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = "+i)

Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = HH
Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = HV
Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = VH
Overpass Duration: 2020-02-01 02:13:16 - 2020-02-01 02:15:58 (UTC), DEM used = Intermap Elevation Model DTM, Polarization = VV


In [ ]:
# Close the dataset and end the database session
dataset.close()
session.close()